In [1]:
library(targets)
library(tarchetypes)
library(tidyverse)
library(Seurat)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Attaching SeuratObject

Attaching sp



In [2]:
combined = qs::qread('../01_milo/_targets/objects/combined_pldf_obob5v5')
head(combined)

,polar_label
,<chr>
TCAGGTAAGCACCTGC_1_1,a1_Tany__Astrocytes.none
CTACAGACAACTTGCA_1_1,a1_Tany__Astrocytes.none
GCAGCTGAGTCACTAC_1_1,a1_Tany__Astrocytes.none
AGAGCAGGTCTGATAC_1_1,a1_Tany__Astrocytes.none
TTTGGTTTCCCTCTCC_1_1,a1_Tany__Astrocytes.none
TGTCCCAAGCATTGTC_1_1,a1_Tany__Astrocytes.none


In [3]:
combined %>% group_by(polar_label) %>% 
summarise(n = n()) %>%
arrange(n) %>% head(20)

polar_label,n
<chr>,<int>
NFOL.pos,51
NFOL.neg,65
Unassigned2__Rgs16_Vip.none,147
Microglia.pos,152
b2_Tany.pos,174
b2_Tany.neg,181
Fibroblasts2__Fibroblasts3.none,183
Oxt__Unassigned1.none,185
Mural_Cells2.none,205


In [4]:
combined %>% group_by(polar_label) %>% dplyr::filter(stringr::str_detect(polar_label, 'NFOL')) %>% summarise(n = n()) %>%
arrange(n)

polar_label,n
<chr>,<int>
NFOL.pos,51
NFOL.neg,65
NFOL.none,338


In [5]:
combined %>% group_by(polar_label) %>% dplyr::filter(stringr::str_detect(polar_label, 'Unassigned2')) %>% summarise(n = n()) %>%
arrange(n)

polar_label,n
<chr>,<int>
Unassigned2__Rgs16_Vip.none,147
Unassigned2__Unassigned1.none,507
Unassigned2.none,12660


In [6]:
combined %>% group_by(polar_label) %>% dplyr::filter(stringr::str_detect(polar_label, 'Microglia')) %>% summarise(n = n()) %>%
arrange(n)

polar_label,n
<chr>,<int>
Microglia.pos,152
Microglia.neg,284
NG2_OPC2__Microglia.none,504
Microglia.none,856


In [7]:
combined %>% group_by(polar_label) %>% dplyr::filter(stringr::str_detect(polar_label, 'Microglia')) %>% summarise(n = n()) %>%
arrange(n)

polar_label,n
<chr>,<int>
Microglia.pos,152
Microglia.neg,284
NG2_OPC2__Microglia.none,504
Microglia.none,856


In [8]:
combined = combined %>% 
mutate(polar_label = case_when(
    stringr::str_detect(polar_label, 'NFOL') ~ 'NFOL',
    TRUE ~ polar_label
))
combined %>% group_by(polar_label) %>% 
summarise(n = n()) %>%
arrange(n) %>% head(20)

polar_label,n
<chr>,<int>
Unassigned2__Rgs16_Vip.none,147
Microglia.pos,152
b2_Tany.pos,174
b2_Tany.neg,181
Fibroblasts2__Fibroblasts3.none,183
Oxt__Unassigned1.none,185
Mural_Cells2.none,205
a2_Tanycytes__b1_Tanycytes.pos,208
NG2_OPC2.pos,263


In [9]:
exp_other = qs::qread('../00_preprocessing/_targets/objects/exp_labelled_other')
exp_other

An object of class Seurat 
56583 features across 37179 samples within 3 assays 
Active assay: SCT (24262 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [27]:
exp_other = AddMetaData(exp_other, metadata = combined)
exp_other@meta.data = exp_other@meta.data %>%
  mutate(polar_label = ifelse(is.na(polar_label), stringr::str_replace(labels, '-', '__') %>% paste0('.none'), polar_label)) 

,orig.ident,nCount_RNA,nFeature_RNA,Index.10x,nCount_HTO,nFeature_HTO,Index.HTO,nCount_spliced,nFeature_spliced,nCount_unspliced,⋯,strain,treatment,time,collection_date,isolation_date,batch,predicted.id,prediction.score.max,labels,polar_label
,<chr>,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,<int>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<dbl>,<fct>,<chr>
CCTATCGAGCCATTCA_1_1,SI-TT-A11,49695,7017,SI-TT-A11,1473,12,D706,7105,3206,42590,⋯,obob,FGF1,Day14,220407,22/6/2022,Batch 1,NG2_OPC2,0.3917772,NG2_OPC2-Microglia,NG2_OPC2__Microglia.none
GTCCTCAAGCGAGGAG_1_1,SI-TT-A11,41375,6414,SI-TT-A11,2757,12,D706,6392,2838,34983,⋯,obob,FGF1,Day5,211213,22/6/2022,Batch 1,NFOL,1.0000000,NFOL,NFOL
AAGGTAATCTGAGGCC_1_1,SI-TT-A11,40000,6407,SI-TT-A11,1678,12,D706,5088,2532,34912,⋯,obob,FGF1,Day14,220131,22/6/2022,Batch 1,NG2_OPC2,0.9625983,NG2_OPC2-Microglia,NG2_OPC2__Microglia.none
TCCGTGTCACCTGTCT_1_1,SI-TT-A11,39270,6913,SI-TT-A11,1516,12,D706,6058,2883,33212,⋯,obob,FGF1,Day14,220407,22/6/2022,Batch 1,Parstuber2A,0.2406799,NG2_OPC2-Microglia,NG2_OPC2__Microglia.none
ATGGTTGGTCGATTCA_1_1,SI-TT-A11,38974,6546,SI-TT-A11,1582,12,D706,5709,2734,33265,⋯,BL6,Veh_PF,Day5,220329,22/6/2022,Batch 1,NG2_OPC2,0.5186747,NG2_OPC2-Microglia,NG2_OPC2__Microglia.none
CTTCGGTTCAGCTGAT_1_1,SI-TT-A11,40118,5968,SI-TT-A11,2558,12,D706,7081,2634,33037,⋯,obob,FGF1,Day5,211213,22/6/2022,Batch 1,NFOL,0.9833581,NFOL,NFOL


In [32]:
exp_neuron = qs::qread('../00_preprocessing/_targets/objects/exp_RElabelled_neuron')
exp_neuron

An object of class Seurat 
59227 features across 107464 samples within 3 assays 
Active assay: SCT (26906 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [33]:
exp_neuron = AddMetaData(exp_neuron, metadata = combined)
exp_neuron@meta.data = exp_neuron@meta.data %>%
  mutate(polar_label = ifelse(is.na(polar_label), stringr::str_replace(labels, '-', '__') %>% paste0('.none'), polar_label)) 

In [34]:
head(exp_neuron@meta.data, 20)

,orig.ident,nCount_RNA,nFeature_RNA,Index.10x,nCount_HTO,nFeature_HTO,Index.HTO,nCount_spliced,nFeature_spliced,nCount_unspliced,⋯,strain,treatment,time,collection_date,isolation_date,batch,predicted.id,prediction.score.max,labels,polar_label
,<chr>,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,<int>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
GCAGCCACACAGCCAC_1_1,SI-TT-A11,69641,7356,SI-TT-A11,1532,12,D706,9837,3757,59804,⋯,obob,FGF1,Day5,220329,22/6/2022,Batch 1,Nr5a1_Bdnf,1.0000000,Nr5a1_Bdnf,Nr5a1_Bdnf.none
TGCTTGCAGTTCCGTA_1_1,SI-TT-A11,71825,7455,SI-TT-A11,1616,12,D706,8834,3443,62991,⋯,obob,Veh_PF,Day14,220201,22/6/2022,Batch 1,Unassigned1,0.8065577,Unassigned1.sc17.sc28,Unassigned1.sc17.sc28.none
TGCGGCACAGCATGCC_1_1,SI-TT-A11,65517,7244,SI-TT-A11,2197,12,D706,8779,3410,56738,⋯,BL6,Veh_PF,Day5,220329,22/6/2022,Batch 1,Nr5a1_Bdnf,1.0000000,Nr5a1_Bdnf,Nr5a1_Bdnf.none
GAGGCAAGTACCTAAC_1_1,SI-TT-A11,68817,7344,SI-TT-A11,1884,12,D706,8246,3326,60571,⋯,obob,Veh_PF,Day14,220201,22/6/2022,Batch 1,Unassigned1,0.8525726,Unassigned1.sc17.sc28,Unassigned1.sc17.sc28.none
TAGCACATCACTGTTT_1_1,SI-TT-A11,70863,7560,SI-TT-A11,1771,12,D706,9612,3747,61251,⋯,BL6,Veh_PF,Day5,220329,22/6/2022,Batch 1,Sst_Pthlh,0.6754849,Sst_Pthlh-Ghrh,Sst_Pthlh__Ghrh.none
GTAATGCTCTGTCCCA_1_1,SI-TT-A11,69910,7425,SI-TT-A11,2555,12,D706,10497,3727,59413,⋯,obob,FGF1,Day5,211213,22/6/2022,Batch 1,Hdc,1.0000000,Gm8773_Tac1-Hdc,Gm8773_Tac1__Hdc.none
GATCACACACCCAAGC_1_1,SI-TT-A11,62110,6862,SI-TT-A11,1728,12,D706,9451,3566,52659,⋯,obob,Veh_PF,Day5,220329,22/6/2022,Batch 1,Unassigned1,0.6368759,Unassigned1.sc37,Unassigned1.sc37.none
GGACGTCCAGGCCCTA_1_1,SI-TT-A11,63372,7265,SI-TT-A11,1778,12,D706,8563,3478,54809,⋯,obob,Veh_PF,Day5,211214,22/6/2022,Batch 1,Nr5a1_Bdnf,1.0000000,Nr5a1_Bdnf,Nr5a1_Bdnf.none
CAGATACAGGTCATTC_1_1,SI-TT-A11,63596,7244,SI-TT-A11,1523,12,D706,8690,3530,54906,⋯,obob,FGF1,Day14,220407,22/6/2022,Batch 1,Nr5a1_Bdnf,1.0000000,Nr5a1_Bdnf,Nr5a1_Bdnf.none


In [37]:
exp_neuron@meta.data %>% pull(polar_label) %>% unique %>% length

[1] 56

In [38]:
exp_other@meta.data %>% pull(polar_label) %>% unique %>% length

[1] 36

In [40]:
exp_neuron@meta.data %>% pull(polar_label) %>% unique %>% sort

[1] "a2_Tanycytes__b1_Tanycytes.none" "Agrp.neg"                       
 [3] "Agrp.none"                       "Agrp.pos"                       
 [5] "Ghrh.neg"                        "Ghrh.none"                      
 [7] "Ghrh.pos"                        "Gm8773_Tac1__Hdc.none"          
 [9] "Gpr50.none"                      "Htr3b.neg"                      
[11] "Htr3b.none"                      "Htr3b.pos"                      
[13] "Kisspeptin.none"                 "Nfix_Htr2c.none"                
[15] "NG2_OPC2__Microglia.none"        "Nr5a1_Bdnf__Nr5a1_Nfib.none"    
[17] "Nr5a1_Bdnf__Unassigned1.none"    "Nr5a1_Bdnf.none"                
[19] "Nr5a1_Nfib.none"                 "Oxt__Unassigned1.none"          
[21] "Pomc_Glipr1__Slc17a6_Trhr.neg"   "Pomc_Glipr1__Slc17a6_Trhr.none" 
[23] "Pomc_Glipr1__Slc17a6_Trhr.pos"   "Pomc_Lepr.none"                 
[25] "Qrfp.none"                       "Rgs16_Dlx1__Unassigned2.none"   
[27] "Rgs16_Dlx1.neg"                  "Rgs16_Dlx1.none"                
[29] "Rgs16_Nmu.none"                  "Slc17a6_Fam19a2.none"           
[31] "Slc17a6_Trhr.neg"                "Slc17a6_Trhr.none"              
[33] "Sst_Nts.none"                    "Sst_Pthlh__Ghrh.none"           
[35] "Sst_Pthlh.sc21.none"             "Sst_Pthlh.sc23.neg"             
[37] "Sst_Pthlh.sc23.none"             "Sst_Unc13c__Agrp.neg"           
[39] "Sst_Unc13c__Agrp.none"           "Sst_Unc13c__Agrp.pos"           
[41] "Tbx19.none"                      "Th_Nfib__Arx_Nr5a2.none"        
[43] "Th_Slc6a3.none"                  "Tmem215.neg"                    
[45] "Tmem215.none"                    "Trh_Cxcl12.none"                
[47] "Trh_Lef1.neg"                    "Trh_Lef1.none"                  
[49] "Trh_Lef1.pos"                    "Unassigned1.sc17.sc28.none"     
[51] "Unassigned1.sc17.sc28.pos"       "Unassigned1.sc37.none"          
[53] "Unassigned1.sc39.none"           "Unassigned2__Rgs16_Vip.none"    
[55] "Unassigned2__Unassigned1.none"   "Unassigned2.none"

In [41]:
exp_other@meta.data %>% pull(polar_label) %>% unique %>% sort

[1] "a1_Tany__Astrocytes.neg"         "a1_Tany__Astrocytes.none"       
 [3] "a1_Tany__Astrocytes.pos"         "a2_Tanycytes__b1_Tanycytes.neg" 
 [5] "a2_Tanycytes__b1_Tanycytes.none" "a2_Tanycytes__b1_Tanycytes.pos" 
 [7] "a2_Tanycytes.neg"                "a2_Tanycytes.none"              
 [9] "a2_Tanycytes.pos"                "Astrocytes.neg"                 
[11] "Astrocytes.none"                 "Astrocytes.pos"                 
[13] "b1_Tanycytes.neg"                "b1_Tanycytes.none"              
[15] "b1_Tanycytes.pos"                "b2_Tany.neg"                    
[17] "b2_Tany.none"                    "b2_Tany.pos"                    
[19] "Endothelial.none"                "Epend.none"                     
[21] "Fibroblasts2__Fibroblasts3.none" "Microglia.neg"                  
[23] "Microglia.none"                  "Microglia.pos"                  
[25] "MOL__a2_Tanycytes.none"          "MOL.neg"                        
[27] "MOL.none"                        "MOL.pos"                        
[29] "Mural_Cells2.none"               "NFOL"                           
[31] "NG2_OPC2__Microglia.none"        "NG2_OPC2.neg"                   
[33] "NG2_OPC2.none"                   "NG2_OPC2.pos"                   
[35] "Parstuber2A__ParsTuber1.none"    "Tbx19.none"

In [48]:
exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_ends("Rik") %>% `!` %>% which

[1]     1     2     3     4     5     6     7     8     9    10    11    12
   [13]    13    14    15    16    17    18    19    21    22    23    24    25
   [25]    26    27    28    29    30    31    33    34    35    36    37    38
   [37]    39    40    41    42    44    45    46    47    48    49    50    51
   [49]    52    53    54    55    56    57    58    59    60    61    62    63
   [61]    64    65    66    67    68    69    70    71    72    73    74    75
   [73]    76    77    78    79    80    81    82    84    85    86    87    88
   [85]    89    90    91    92    93    94    95    96    97    98    99   100
   [97]   101   102   103   104   105   106   107   108   109   110   111   112
  [109]   113   114   115   116   117   118   119   120   121   122   123   124
  [121]   125   126   127   128   129   130   131   132   133   134   135   137
  [133]   138   139   140   141   142   143   144   145   146   147   148   149
  [145]   150   151   152   153   154   155   156   157   158   160   163   164
  [157]   165   166   167   168   169   170   171   172   173   174   175   176
  [169]   177   178   179   180   181   182   183   184   185   186   187   188
  [181]   189   190   191   192   193   194   195   197   198   199   200   201
  [193]   202   203   204   205   206   207   208   209   210   211   212   213
  [205]   214   215   216   217   218   219   220   221   222   223   224   225
  [217]   226   227   228   229   230   231   232   233   234   235   237   239
  [229]   240   241   242   243   244   245   246   247   248   249   250   251
  [241]   252   253   254   255   256   257   258   259   260   261   262   263
  [253]   264   265   266   267   268   269   270   271   272   273   274   275
  [265]   276   277   278   279   280   281   282   283   284   286   287   288
  [277]   289   290   291   292   293   294   295   296   297   298   299   301
  [289]   302   303   304   305   307   308   310   312   313   314   315   316
  [301]   317   318   319   320   321   322   323   324   325   326   327   328
  [313]   329   330   331   332   333   334   335   336   337   338   339   340
  [325]   341   343   344   345   346   347   348   349   350   351   352   353
  [337]   354   355   356   357   358   359   360   361   362   363   364   365
  [349]   366   367   368   369   370   371   372   373   374   375   376   377
  [361]   378   379   380   381   382   383   384   385   386   387   388   389
  [373]   390   392   393   394   396   397   398   399   400   401   402   403
  [385]   404   405   406   407   408   409   410   411   412   413   414   415
  [397]   416   418   419   420   421   422   423   425   426   427   428   429
  [409]   430   431   432   433   434   435   436   437   439   440   441   442
  [421]   443   444   445   446   447   448   449   450   451   452   453   454
  [433]   455   456   457   458   459   460   461   462   463   464   465   466
  [445]   467   468   469   470   471   472   473   474   475   476   477   478
  [457]   479   480   481   482   483   484   485   486   487   488   489   490
  [469]   491   492   493   494   495   496   497   498   499   500   501   502
  [481]   503   504   505   506   507   508   509   510   511   512   513   514
  [493]   515   516   517   518   519   520   521   522   523   524   525   526
  [505]   527   528   529   530   531   532   533   534   535   536   537   538
  [517]   539   540   542   543   544   545   546   547   548   549   550   551
  [529]   552   553   554   555   556   557   558   559   560   561   562   563
  [541]   564   565   566   567   568   569   570   571   572   573   574   575
  [553]   576   577   578   580   581   582   583   584   585   586   587   588
  [565]   589   590   591   593   594   596   597   598   599   600   601   602
  [577]   603   604   605   606   607   608   609   610   611   612   613   614
  [589]   615   616   617   618   619   620   621   622   623   624   625   626
  [6

In [49]:
exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_starts("ENSMUSG") 

[1]  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
   [13] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
   [25] FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE
   [37] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE  TRUE
   [49] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE
   [61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
   [73]  TRUE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
   [85] FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE  TRUE FALSE FALSE  TRUE
   [97]  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE
  [109] FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE
  [121] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [133] FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE
  [145] FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE
  [157] FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [169] FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE
  [181]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
  [193] FALSE FALSE  TRUE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE
  [205] FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE
  [217] FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE
  [229] FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE
  [241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [253] FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE
  [265]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE
  [277] FALSE  TRUE FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE
  [289] FALSE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE
  [301]  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [313] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE
  [325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [337] FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE  TRUE
  [349] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [361] FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [397] FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE
  [409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [421]  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE FALSE FALSE
  [433]  TRUE  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE
  [445]  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [457] FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [469]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [481]  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [493] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE
  [505] FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE
  [517] FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [529]  TRUE  TRUE FALSE FALSE  TRUE FALSE FALSE  TRUE  TRUE FALSE  TRUE FALSE
  [541] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [553] FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [565] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE
  [577]  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE
  [589] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE
  [6

In [97]:
not_a_rik = exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_ends("Rik") %>% `!`
not_an_ensmusg = exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_starts("ENSMUSG") %>% `!`
not_a_Gm = exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_starts("Gm\\d") %>% `!`
not_4_or_more_digits = exp_other@assays$RNA@meta.features %>% rownames %>% stringr::str_ends("\\d{4,}") %>% `!`
only_named_genes = (not_a_rik & not_an_ensmusg & not_a_Gm & not_4_or_more_digits) %>% which
only_named_genes

[1]     3     4     5     6     7     8     9    10    12    13    14    16
   [13]    17    18    19    22    25    27    28    29    33    34    37    39
   [25]    40    41    42    44    46    47    49    50    51    52    53    57
   [37]    58    59    61    62    63    64    65    66    67    68    69    70
   [49]    74    76    78    79    80    82    84    85    86    87    88    91
   [61]    92    94    98    99   100   101   102   104   105   106   107   108
   [73]   109   110   111   112   114   119   120   121   124   125   126   127
   [85]   128   130   131   132   133   134   139   141   142   143   145   146
   [97]   147   149   150   151   153   154   155   157   158   163   164   165
  [109]   166   167   168   169   170   171   173   174   175   176   180   182
  [121]   184   185   186   187   188   189   190   191   193   194   199   200
  [133]   201   202   204   205   207   210   213   214   215   216   217   218
  [145]   219   225   226   227   228   230   231   232   234   235   240   241
  [157]   242   243   244   245   246   247   248   249   250   251   252   253
  [169]   254   255   256   259   260   261   262   263   266   267   268   269
  [181]   270   271   272   273   276   277   279   280   281   286   287   288
  [193]   289   291   296   297   298   299   304   305   307   308   310   312
  [205]   313   314   316   317   318   319   321   322   323   324   325   326
  [217]   327   328   329   330   331   332   333   334   335   336   337   341
  [229]   343   344   347   349   351   352   353   354   355   356   357   358
  [241]   359   360   361   362   364   367   368   369   370   371   372   373
  [253]   374   375   376   378   379   380   381   382   384   385   386   387
  [265]   388   389   390   392   393   394   397   399   401   404   405   406
  [277]   407   409   410   411   412   413   414   415   418   420   429   431
  [289]   432   435   437   439   440   444   446   449   450   451   452   453
  [301]   454   456   457   458   459   461   464   465   466   467   468   470
  [313]   471   472   473   474   475   476   477   478   479   480   482   483
  [325]   484   486   487   488   489   490   491   492   493   494   495   496
  [337]   500   501   502   503   504   505   506   507   509   510   511   512
  [349]   514   516   517   519   521   522   523   524   525   526   527   528
  [361]   531   532   534   538   540   543   544   545   546   547   548   549
  [373]   550   552   553   554   561   562   563   564   565   566   567   568
  [385]   569   570   571   572   573   575   576   578   580   581   583   585
  [397]   586   587   589   590   591   593   594   596   600   601   602   603
  [409]   604   605   606   608   610   611   612   615   616   617   618   619
  [421]   620   621   622   623   624   629   632   633   634   636   638   641
  [433]   642   643   644   646   647   648   649   651   652   656   657   658
  [445]   663   664   665   666   667   668   669   670   671   672   673   674
  [457]   675   676   677   678   679   680   681   683   684   685   686   688
  [469]   690   691   692   693   694   695   696   697   698   699   701   702
  [481]   703   704   705   706   707   708   709   710   711   712   713   714
  [493]   715   716   717   718   720   721   722   723   726   727   731   732
  [505]   733   734   735   736   737   738   740   742   743   744   745   747
  [517]   748   749   750   751   754   756   757   758   760   762   763   764
  [529]   766   767   769   772   779   780   781   782   788   790   792   794
  [541]   796   799   802   805   806   807   811   812   813   814   815   816
  [553]   819   822   825   826   827   829   831   832   834   835   836   839
  [565]   842   843   844   845   846   847   848   849   850   851   852   853
  [577]   854   855   856   857   859   860   861   862   863   868   869   870
  [589]   872   873   874   876   878   879   881   882   883   884   885   888
  [6

In [105]:
.get_only_named_genes = function(exp){
    not_a_rik = exp@assays$RNA@meta.features %>% rownames %>% stringr::str_ends("Rik") %>% `!`
    not_an_ensmusg = exp@assays$RNA@meta.features %>% rownames %>% stringr::str_starts("ENSMUSG") %>% `!`
    not_a_Gm = exp@assays$RNA@meta.features %>% rownames %>% stringr::str_starts("Gm\\d") %>% `!`
    not_4_or_more_digits = exp@assays$RNA@meta.features %>% rownames %>% stringr::str_ends("\\d{4,}") %>% `!`
    only_named_genes = (not_a_rik & not_an_ensmusg & not_a_Gm & not_4_or_more_digits) %>% which
    only_named_genes
}

make_new_seurat_ngo = function(exp){
    counts_exp = GetAssayData(exp, slot="counts", assay="RNA") 
    counts_exp = counts_exp[get_only_named_genes(exp),]
    new_exp = CreateSeuratObject(counts=counts_exp, meta.data = exp@meta.data)
    new_exp
}

In [102]:
only_named_genes = exp_other@assays$RNA@meta.features %>% rownames %>% `[`(only_named_genes)

In [107]:
counts_other <- GetAssayData(exp_other, slot="counts", assay="RNA") 
counts_other <- counts_other[get_only_named_genes(exp_other),]
counts_neuron <- GetAssayData(exp_neuron, slot="counts", assay="RNA") 
counts_neuron <- counts_neuron[get_only_named_genes(exp_neuron),]

In [ ]:
new_other <- CreateSeuratObject(counts=counts_other, meta.data = exp_other@meta.data)
new_neuron <- CreateSeuratObject(counts=counts_other, meta.data = exp_neuron@meta.data)